### Comparison of Algorihms 2 and 3
#### Used for Figure 7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from clawpack import pyclaw
import sys
from piston_Lagrangian import setup
from matplotlib import animation
from IPython.display import HTML
plt.rcParams['text.usetex'] = True
plt.rcParams['animation.ffmpeg_path'] = r'/usr/local/bin/ffmpeg'
import itertools
from utils import max_error

In [ ]:
def generate_input_CLAW(method, order_relaxation):
    far_field_damping = False #Exponential far_field_damping (w. source term)
    scalar_far_field =False
    relaxation_method = False
    matrix_filter=False
    integral_source = False
    euler_RS = 'euler'
    solver_class = 'sharpclaw_custom'
    Strang = False
    before_stage = False

    if method == "RM":
        relaxation_method = True 
        
    elif method == "RM-M":
        relaxation_method = True
        matrix_filter = True

    else:
        print("###############################")
        print("Method not implemented")
        print("###############################")
        return 1
    
    if order_relaxation ==0: #Relaxation after each RK stage
            before_stage = True
    if order_relaxation ==2:
            Strang = True

    return (euler_RS, relaxation_method, far_field_damping,
             matrix_filter, scalar_far_field, integral_source,
             solver_class, Strang, before_stage)

In [ ]:
def set_and_run_CLAW(method, order_relaxation, data, adaptiveRM=False):
    (tfinal,xmax2,mx2,piston_freq,M,
    start_slowing,stop_slowing,
    start_absorbing,stop_absorbing) = data
    (euler_RS, relaxation_method, far_field_damping,
    matrix_filter, scalar_far_field,
    integral_source, solver_class,
    Strang, before_stage) = generate_input_CLAW(method, order_relaxation=order_relaxation)
    piston_problem2 = setup(tfinal=tfinal, xmax=xmax2, mx=mx2, M=M, CFL=0.8,limiting=1,nout=100,
                        solver_type=solver_class, order=2, time_integrator='Heun',
                        euler_RS=euler_RS, 
                        relaxation_method=relaxation_method,
                        far_field_damping=far_field_damping,
                        matrix_filter=matrix_filter,
                        start_slowing=start_slowing, stop_slowing=stop_slowing,
                        start_absorbing=start_absorbing, stop_absorbing=stop_absorbing,
                        piston_freq=piston_freq,
                        sigma_damping_type="Mayer", 
                        sigma_slowing_type="Engsig-Karup2",
                        scalar_far_field=scalar_far_field,
                        implicit_integrator=True,
                        integral_source=integral_source,
                        before_stage=before_stage,
                        adaptiveRM=adaptiveRM,
                        Strang=Strang) #Not really using implicit integrator at the moment
    piston_problem2.verbosity = 0
    piston_problem2.run()
    piston_problem2.solver.__del__()
    return piston_problem2

In [ ]:
def comparison_setup(sponge_layer_width_list,piston_freq=1., p=20, p_domain=30, p_ABC=20, p_domain_ABC=30, M=0.4, 
                    verbose=True, N=200, adaptiveRM=False):
    if verbose:
        print("#################################################################")
        print("RUNNING SIMULATION WITH PARAMETERS:\npiston_freq = ", piston_freq,
         ", p = ", p, ", p_domain = ", p_domain, ", M = ", M)

    #Reference solution
    xmax1 = p_domain*2*np.pi #p*2*np.pi
    #N = 200 #Cells per period at final time
    mx1 = p_domain*N #Number of grid points
    tfinal = p*2*np.pi

    #Run reference solution
    piston_problem1 = setup(tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw',
    order=2, time_integrator='Heun', nout=100, piston_freq=piston_freq)
    piston_problem1.verbosity = 0
    piston_problem1.run()
    piston_problem1.solver.__del__()

    #Run "truncated" solutions
    xmax2 = p_domain_ABC*2*np.pi #p*2*np.pi
    #N = 200 #Cells per period at final time
    mx2 = p_domain_ABC*N #Number of grid points

    method_list = ["RM","RM-M"]
    orders_relaxation = [1,2,0]
    #Combine all methods in tuples
    method_list = list(itertools.product(method_list,orders_relaxation))
    method_list_names = [str(met[0])+"-"+str(met[1]) for met in method_list]
    

    start_slowing = p_ABC*np.pi#p_ABC*2*np.pi
    start_absorbing = p_ABC*np.pi#p_ABC*2*np.pi

    error_list=[]
    for width in sponge_layer_width_list:
        if verbose:
            print("Layer width = ", width)
        stop_slowing = start_absorbing + width*2*np.pi
        stop_absorbing = start_absorbing + width*2*np.pi
        data = (tfinal,xmax2,mx2,piston_freq,M,
            start_slowing,stop_slowing,
            start_absorbing,stop_absorbing) 

        x_comparison = min(start_slowing,start_absorbing)
        error_list.append([])
        for method in method_list:
            met = method[0]
            order_relaxation = method[1]
            if verbose:
                print("Running ", met, " with order ", order_relaxation)
            piston_problem2 = set_and_run_CLAW(met,order_relaxation, data, adaptiveRM=adaptiveRM)
            indx = np.where(piston_problem2.grid.x.centers<=x_comparison)
            error_list[-1].append(max_error(piston_problem1,piston_problem2,indx=indx))
            if not np.allclose(piston_problem1.grid.x.centers[indx],piston_problem2.grid.x.centers[indx]):
                raise ValueError("Grids are not the same!!!")
            if verbose:
                if order_relaxation>0:
                    print(met," with order ", order_relaxation, " DONE")
                else:
                    print(met," with relaxation after each RK stage DONE")
                
    if verbose:
        print("#################################################################")
    return np.array(error_list)

# Running comparison with NO adaptiveRM

In [ ]:
#list_sponge_layer_widths = np.round(np.logspace(np.log10(0.001),np.log10(2.),10),3)
list_sponge_layer_widths = np.round(np.logspace(np.log10(0.01),np.log10(10.),11),2)

In [ ]:
N_per_period = [10, 50, 250]
#list_error_arrays04 = [[] for i in range(len(N_per_period))]
M = 0.4

for N in N_per_period:
    error_list = comparison_setup(sponge_layer_width_list=list_sponge_layer_widths, M=M,
                                piston_freq=1., p=20, p_domain=30, p_ABC=20, p_domain_ABC=30, N=N)
    print("#################################################################")
    print("Error list: ",error_list)
    print("#################################################################")
    np.savetxt("./data_First_Second/N="+str(N)+".txt",error_list)

In [ ]:
N_per_period = [10, 50, 250]
list_error_arrays= [[] for i in range(len(N_per_period))]

for i,N in enumerate(N_per_period):
    list_error_arrays[i] = np.loadtxt("./data_First_Second/N="+str(N)+".txt")

list_titles = ["N = "+str(N) for N in N_per_period]

In [ ]:
import matplotlib as mpl
def plot_errors_diff_Nx(list_error_arrays, list_titles,sponge_layer_width_list, ylim=None, list_ylim=None, logscales=[]):
    #Create a figure with len(error_list_np) axes in a column
    plt.close()
    fig,ax = plt.subplots(len(list_error_arrays),1,figsize=(5,2*len(list_error_arrays)),dpi=300)

    #######################
    #Sorry for hardcoding this
    #Self-convergence error (Self_convergence.ipynb)
    convError = np.array([1.061677543091432, 0.16727682415561385, 0.030993006342411783])
    #Constant extrapolation error (Constant_Extrapolation_Error.ipynb)
    constExtrapError = np.array([0.02572953158743871, 0.022003876484894457, 0.03087486484936671])
    #######################

    method_list = ["RM","RM-M"]
    orders_relaxation = [1,2,0]
    #Combine all methods in tuples
    method_list = list(itertools.product(method_list,orders_relaxation))
    method_list_names = [str(met[0])+"-"+str(met[1]) for met in method_list]
    method_list = ['RM ', 'RM 2', 'RM RK', 'RM-M ', 'RM-M 2', 'RM-M RK']
    #method_list = ["RM","RM+SO","RM+M","RM+SO","SDO","SDO scalar", "SDO integral"]
    color_list = ['blue','red','green','orange','purple','brown']#, 'black']
    linetype_list = ['solid','dashed','dotted','solid','dashed',"dotted"]#, "dotted"]
    marker_list = ['o','o','o','x','x','x']
    for i,error_list_np in enumerate(list_error_arrays):
        ax[i].set_xscale('log')
        if i<len(logscales):
            if logscales[i]:
                ax[i].set_yscale('log')
        #remove all minor ticks in x and y axis
        ax[i].xaxis.set_minor_locator(mpl.ticker.NullLocator())
        ax[i].yaxis.set_minor_locator(mpl.ticker.NullLocator())
        # Set the tick locations to be the values from the sponge_layer_width_list
        if i==len(list_error_arrays)-1:
            ax[i].set_xticks(sponge_layer_width_list)
            xtickslabels = np.round(sponge_layer_width_list,2)
            xtickslabels[:3] = [f"{width:.2e}" for width in sponge_layer_width_list[:3]]
            ax[i].set_xticklabels(xtickslabels)
            ax[i].xaxis.set_tick_params(which='minor', bottom=False)
        #Forcing the x-axis to be between 0.23 and 11
        ax[i].set_xlim(sponge_layer_width_list[0],sponge_layer_width_list[-1])
        if ylim is not None:
            ax[i].set_ylim(ylim)
        if list_ylim is not None:
            ax[i].set_ylim(list_ylim[i])
        # Remove the unwanted ticks
        
        ax[i].set_xlabel("Sponge layer length/wavelength "+r'($\omega/L$)')

        ax[i].axvline(x=1,linestyle='--',color='black',linewidth=0.5)
        for j,method in enumerate(method_list):
            ax[i].plot(sponge_layer_width_list,error_list_np[:,j],color=color_list[j],linewidth=2, linestyle=linetype_list[j],label=method)
            ax[i].scatter(sponge_layer_width_list,error_list_np[:,j],color=color_list[j],marker=marker_list[j] )
            #Connect the points with a thin line    
        if i==1:
            ax[i].set_ylabel("Error ABC")
        # if i==0:
        #     ax[i].legend()
        ax[i].set_title(list_titles[i])
        hline1, =ax[i].plot(sponge_layer_width_list,
                        convError[i]*np.ones_like(sponge_layer_width_list),
                        linestyle='-.',
                        color='black',marker='^',label="Discretization Error")
        #hline2 = ax[i].hlines(constExtrapError[i],0.01,5,linestyles=':',colors='grey',
                     #label="Constant Extrapolation")
        hline2, =ax[i].plot(sponge_layer_width_list,
                      constExtrapError[i]*np.ones_like(sponge_layer_width_list),
                      linestyle=':',
                      color='grey',marker='*',label="Const. Extrap. Error")   
    plt.show()

In [ ]:
plot_errors_diff_Nx(list_error_arrays, list_titles=["N=10","N=50","N=250"], sponge_layer_width_list=list_sponge_layer_widths, ylim=None, list_ylim=None, logscales=[True,True,True])

# Running comparison with adaptiveRM

In [ ]:
# list_sponge_layer_widths = np.round(np.logspace(np.log10(0.01),np.log10(5.),10),3)

In [ ]:
# N_per_period = [10, 50, 250]
# #list_error_arrays04 = [[] for i in range(len(N_per_period))]
# list_error_arrays= [[] for i in range(len(N_per_period))]
# M = 0.4

# for i,N in enumerate(N_per_period):
#     error_list = comparison_setup(sponge_layer_width_list=list_sponge_layer_widths, M=M,
#                                 piston_freq=1., p=20, p_domain=30, p_ABC=20, p_domain_ABC=30, N=N)
#     print("#################################################################")
#     print("Error list: ",error_list)
#     print("#################################################################")
#     np.savetxt("./data_First_Second_adaptive/N="+str(N)+".txt",error_list)
#     list_error_arrays[i] = error_list


In [ ]:
# N_per_period = [10, 50, 250]
# list_error_arrays= [[] for i in range(len(N_per_period))]

# for i,N in enumerate(N_per_period):
#     list_error_arrays[i] = np.loadtxt("./data_First_Second_adaptive/N="+str(N)+".txt")

# list_titles = ["N = "+str(N) for N in N_per_period]

In [ ]:
# plot_errors_diff_Nx(list_error_arrays, list_titles=["N=10","N=50","N=250"], sponge_layer_width_list=list_sponge_layer_widths, ylim=None, list_ylim=None, logscales=[True,True,True])